# Geospatial Data Science Tutorial for IC2S2'23
Github: https://github.com/NERDSITU/gdstutorial  
Organizers: https://nerds.itu.dk/
# 1. DATA HANDLING: Data formats, CRS, libraries


This notebook gives an introduction of geospatial data science basics to computational social scientists, including:
- Geospatial data formats
- Coordinate reference systems (CRS)
- Basic Python libraries: geopandas, shapely

This notebook was adapted from:
* Analyze Geospatial Data in Python: GeoPandas and Shapely: https://www.learndatasci.com/tutorials/geospatial-data-python-geopandas-shapely/
* Automating GIS-Processes: https://autogis-site.readthedocs.io/en/latest/lessons/lesson-1/geometry-objects.html
* [Shapely-documentation](https://shapely.readthedocs.io/en/stable/manual.html) 
* [Westra E. (2013), Chapter 3](https://www.packtpub.com/application-development/python-geospatial-development-second-edition)
* A course on Geographic Data Science: https://darribas.org/gds_course/content/bC/lab_C.html
* Automating GIS-Processes: https://autogis-site.readthedocs.io/en/latest/notebooks/L2/02-projections.html
* A course on Geographic Data Science: https://darribas.org/gds_course/content/bD/lab_D.html


## Imports

In [ ]:
import os
os.environ['USE_PYGEOS'] = '0' # Temporary fix to use the brand-new shapely 2.0

import geopandas as gpd # for geospatial data handling
import osmnx # for handling data from OpenStreetMap (osm) with the help of networkX (nx)

import pandas as pd
import contextily as cx # for plotting
import matplotlib.pyplot as plt # for plotting

In this lecture, we will learn how to **load, manipulate** and **visualize** spatial data with a focus on *vector* data. 

In some senses, spatial data are usually included simply as "one more column" in a table. However, *spatial is special* sometimes and there are few aspects in which geographic data differ from standard numerical tables. Dealing with spatial data in Python largely resembles dealing with non-spatial data.

## Shapely

Python has a specific module called [Shapely](https://shapely.readthedocs.io/en/stable/manual.html) for doing various geometric operations. 

Shapely is the standard for working with geospatial data in Python.

Shapely builds on widely used standards: Shapely concepts applies generally.

Basic knowledge of using Shapely is fundamental for understanding how geometries are stored and handled in GeoPandas (we will come back to GeoPandas next week).

The most fundamental geometric objects are `Points`, `Lines` and `Polygons` which are the basic ingredients when working with spatial data in vector format (regardless of programming language and library).

![Spatial data model](files/SpatialDataModel.png)

*Fundamental geometric objects that can be used in Python with* [Shapely](https://shapely.readthedocs.io/en/stable/manual.html).

**Geometric objects in Shapely consist of coordinate tuples where:**

-  `Point` -object represents a single point in space. Points can be either two-dimensional (x, y) or three dimensional (x, y, z).
-  `LineString` -object (i.e. a line) represents a sequence of points joined together to form a line. Hence, a line consist of a list of at least two coordinate tuples
-  `Polygon` -object represents a filled area that consists of a list of at least three coordinate tuples that forms the outerior ring and a (possible) list of hole polygons.

**It is also possible to have a collection of geometric objects (e.g. Polygons with multiple parts):**

-  `MultiPoint` -object represents a collection of points and consists of a list of coordinate-tuples
-  `MultiLineString` -object represents a collection of lines and consists of a list of line-like sequences
-  `MultiPolygon` -object represents a collection of polygons that consists of a list of polygon-like sequences that construct from exterior ring and (possible) hole list tuples

**Useful attributes and methods in Shapely include:**

-  Creating lines and polygons based on a collection of point objects.
-  Calculating areas/length/bounds etc. of input geometries
-  Conducting geometric operations based on the input geometries such as `union`, `difference`, `distance` etc.
-  Conducting spatial queries between geometries such as `intersects`, `touches`, `crosses`, `within` etc.


In [ ]:
# Import necessary geometric objects from shapely module
from shapely.geometry import Point, LineString, Polygon, MultiPoint

### Points and lines

In [ ]:
# Create Point geometric objects with coordinates
point1 = Point(2.2, 4.2)
point2 = Point(7.2, -25.1)
point3 = Point(9.26, -2.456)

# Create a MultiPoint object of our points 1,2 and 3
multi_point = MultiPoint([point1, point2, point3])

# Create a LineString from our Point objects
line = LineString([point1, point2, point3])

Let's see what these variables now contain: 

In [ ]:
point1

In [ ]:
multi_point

In [ ]:
line

Let's see their data types:

In [ ]:
(point1.geom_type, point3D.geom_type, multi_point.geom_type, line.geom_type)

In [ ]:
(type(point1), type(point3D), type(multi_point), type(line))

#### Coordinates

Points and other shapely objects have useful built-in [attributes and methods](https://shapely.readthedocs.io/en/stable/manual.html#general-attributes-and-methods). Using the available attributes, we can for example extract the coordinate values of a Point and calculate the Euclidian distance between points.

Extracting the coordinates of a Point can be done in a couple of different ways:

`coords` attribute contains the coordinate information as a `CoordinateSequence` which is another data type related to Shapely.

In [ ]:
# Get xy coordinate tuple
print(list(point1.coords))
print(list(point3D.coords))
print(list(line.coords))

Objects within multi-objects need to be accessed individually:

In [ ]:
list(multi_point.geoms)

In [ ]:
[list(point.coords) for point in multi_point.geoms]

In [ ]:
list(multi_point.geoms[0].coords)

Here we have coordinate tuples inside lists. Using the attributes `x` and `y` it is possible to fetch the coordinates directly as plain decimal numbers.

In [ ]:
print(point1.x, point1.y)

If you would need to access all x-coordinates or all y-coordinates of the line, you can do it directly using the `xy` attribute: 

In [ ]:
# Extract x and y coordinates separately
xcoords = list(line.xy[0])
ycoords = list(line.xy[1])
print(xcoords)
print(ycoords)

#### Useful point and line functions

It is also possible to calculate the distance between two objects using the [distance](https://shapely.readthedocs.io/en/stable/manual.html#object.distance) method. In our example the distance is calculated in a cartesian coordinate system. When working with real GIS data the distance is based on the used coordinate reference system. always check what is the unit of measurement (for example, meters) in the coordinate reference system you are using.

Let's calculate the distance between `point1` and `point2`:

In [ ]:
# Calculate the distance between point1 and point2
dist = point1.distance(point2)
print(f"Distance between the points is {dist:.2f} units")

It is possible to retrieve specific attributes such as lenght of the line and center of the line (centroid) straight from the LineString object itself:

In [ ]:
# Get the length of the line
l_length = line.length
print(f"Length of our line: {l_length:.2f} units")

In [ ]:
# Get the centroid of the line
print(line.centroid)

As you can see, the centroid of the line is again a Shapely Point object.

### Polygons

Creating a `Polygon`-object continues the same logic of how `Point` and `LineString` were created.

Polygon needs **at least three coordinate-tuples** (three points are required to form a surface):

In [ ]:
# Create a Polygon from the coordinates
poly = Polygon([point1,point2,point3])
poly2 = MultiPoint([point1, point2, point3]).convex_hull

Let's see what our Polygon looks like:

In [ ]:
print(poly)
poly

In [ ]:
print(poly2)
poly2

Geometrically, they are the same, but data-wise they are different because a different point serves as starting and end point!

In [ ]:
poly == poly2

In [ ]:
poly.equals(poly2)

#### Holes

Notice that `Polygon` representation has double parentheses around the coordinates (i.e. `POLYGON ((<values in here>))` ). This is because a Polygon can have *holes* inside of it. 



As the [Polygon](https://shapely.readthedocs.io/en/stable/manual.html#polygons)-docs tell us, a Polygon can be constructed using exterior coordinates and interior coordinates (optional) where the interior coordinates creates a hole inside the Polygon:


Let's see how we can create a `Polygon` with a hole:

In [ ]:
# Define the outer border
border = [(-180, 90), (-180, -90), (180, -90), (180, 90)]

In [ ]:
# Outer polygon
world = Polygon(shell=border)
print(world)

In [ ]:
world

In [ ]:
# Let's create a single big hole where we leave ten units at the boundaries
# Note: there could be multiple holes, so we need to provide list of coordinates for the hole inside a list
hole = [[(-170, 80), (-170, -80), (170, -80), (170, 80)]]
hole

In [ ]:
# Now we can construct our Polygon with the hole inside
frame = Polygon(shell=border, holes=hole)

Let's see what we have now:

In [ ]:
print(frame)
frame

As we can see the `Polygon` has now two different tuples of coordinates. The first one represents the **exterior** and the second one represents the **hole (interior)** inside of the Polygon.

In [ ]:
(frame.exterior, [hole for hole in frame.interiors])

#### Validity

Objects can be constructed via arbitrary coordinates, but must have certain geometric properties to be considered geometrically "valid". Validity can be checked via `is_valid`: 

In [ ]:
print(f"Is world valid?: {world.is_valid}")
print(f"Is frame valid?: {frame.is_valid}")

In [ ]:
polyweird = Polygon([(0,0), (1,1), (1,0), (0,1)])
polyweird

In [ ]:
print(f"Is polyweird valid?: {polyweird.is_valid}")

## GeoPandas

GeoPandas - the spatial extension of pandas - is one of the core libraries for doing geospatial analysis in Python.

There are various different GIS data formats available such as [GPKG](https://en.wikipedia.org/wiki/GeoPackage), [GeoJSON](https://en.wikipedia.org/wiki/GeoJSON), [KML](https://en.wikipedia.org/wiki/Keyhole_Markup_Language), and [Shapefile](https://en.wikipedia.org/wiki/Shapefile). [Geopandas](https://geopandas.org/en/stable/docs/user_guide/io.html) is capable of reading data from all of these formats (plus many more) and combines the capabilities of the data analysis library [pandas](https://pandas.pydata.org/pandas-docs/stable/) with other packages like [shapely](https://shapely.readthedocs.io/en/stable/manual.html) and [fiona](https://fiona.readthedocs.io/en/latest/manual.html) for managing spatial data. 

The main data structures in geopandas are `GeoSeries` and `GeoDataFrame` which extend the capabilities of `Series` and `DataFrames` from pandas. This means that we can use all our pandas skills also when working with geopandas - with some added spatial functionalities 🌐

The main difference between geodataframes and pandas dataframes is that a [geodataframe](https://geopandas.org/en/stable/docs/user_guide/data_structures.html#geodataframe) should contain one column for **geometries**. By default, the name of this column is `'geometry'`. The geometry column is a [geoseries](https://geopandas.org/en/stable/docs/user_guide/data_structures.html#geoseries) which contains the geometries (points, lines, polygons, multipolygons etc.) as shapely objects. More info here: https://geopandas.org/en/stable/getting_started/introduction.html

<img src="files/dataframe.svg">

### Datasets: Load and inspect

To learn these concepts, we will be playing with three main datasets: Cities, streets, and bars in Spain. These datasets can be loaded dynamically from the web, or from your computer.

Regardless of file format, we use the `gpd.read_file()` method. To save files, use `gpd.save_file()`.

#### Cities

We will use an open dataset that contains the boundaries of Spanish cities as *polygons*. We can read it into an object named `cities` by:

In [ ]:
cities = gpd.read_file("https://ndownloader.figshare.com/files/20232174")

The code cell above requires internet connectivity. If you are not online you can read the data from your computer with the following line of code: 

In [ ]:
cities = gpd.read_file("files/cities.gpkg")

We can check that this actually produced a GeoDataFrame....

In [ ]:
type(cities)

...and get a sample of the data like this:

In [ ]:
cities.sample(5)

Now `cities` is a `GeoDataFrame`. Very similar to a traditional, non-spatial `DataFrame`, but with an additional column called `geometry`.


In [ ]:
cities.geometry.head()

**ALWAYS CHECK THE CRS FIRST!**

In [ ]:
cities.crs

How to visualize it:

In [ ]:
cities.plot();

In [ ]:
cities.explore();

In [ ]:
cities.loc[0, 'geometry']

#### Streets

In addition to polygons, we will play with a line layer. For that, we are going to use a subset of street network from the Spanish city of Madrid. 

This dataset is from of a project called "Las calles de las mujeres", a community-driven initiative exploring the extent to which streets are named after women. 

Check out more about the project, including an interactive map at: [https://geochicasosm.github.io/lascallesdelasmujeres/](https://geochicasosm.github.io/lascallesdelasmujeres/)

Read it into an object called `streets` with:

In [ ]:
streets = gpd.read_file("files/streets.geojson")

In [ ]:
streets.head(4)

To explore what type of geometries our geodataframe contains - maybe there's more than one? - we use `geom_type`:

In [ ]:
streets.geom_type.unique()

For this tutorial, we however only want the streets with a Linestring-geometry:

In [ ]:
streets = streets.loc[streets.geom_type=='LineString']
streets.geom_type.unique()

In [ ]:
streets.crs

In [ ]:
streets.loc[2, 'geometry']

In [ ]:
streets.plot();

#### Bars

The final dataset we will rely on is a set of points demarcating the location of bars in Madrid. To obtain it, we will use `osmnx`, a Python library that allows us to query [OpenStreetMap](https://www.openstreetmap.org) (you will learn much more about this later!). Note that we use the method `pois_from_place`, which queries for points of interest (POIs, or `pois`) in a particular place (Madrid in this case). In addition, we can specify a set of tags to delimit the query. We use this to ask _only_ for amenities of the type "bar":

In [ ]:
pois = osmnx.geometries_from_place(
    "Madrid, Spain", tags={"amenity": "bar"}
)

You do not need to know at this point what happens behind the scenes when we run `geometries_from_place` but, if you are curious, we are making a query to [OpenStreetMap](https://www.openstreetmap.org) (almost as if you typed "bars in Madrid, Spain" within Google Maps) and getting the response as a table of data, instead of as a website with an interactive map. Pretty cool, huh?

In [ ]:
pois.crs

In [ ]:
pois.plot();

#### Non-spatial file formats
In all of the above, the data sets were already spatial data formats with defined geometries. It is however also possible to convert a csv with for example x,y-coordinates to a GeoDataFrame, with only a bit more code:

In [ ]:
!head -5 'files/pois_xy.csv'

In [ ]:
df = pd.read_csv('files/pois_xy.csv')

gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.x, df.y) # columns with x and y coordinates in dataframe
)

gdf.head(4)

---

## Manipulating spatial data with GeoPandas

Once we have an understanding of how to explore our spatial data, let us see some of the methods GeoPandas has for working with spatial data. Essentially, the key is to realize that a `GeoDataFrame` (gdf) contains most of its spatial information in a single column named `geometry`, but the rest of it looks and behaves exactly like a non-spatial `DataFrame` (in fact, it is).

GeoDataFrames come with a whole range of traditional GIS operations built-in. Here we will run through a small subset of them that contains some of the most commonly used ones.

### Area calculation

One of the spatial aspects we often need from polygons is their area. "How big is it?" is a question that always haunts us when we think of countries, regions, or cities. To obtain area measurements, first make sure your `GeoDataFrame` is in a *projected* CRS. GeoPandas will issue a warning if you try to do distance or area computations in a geographic CRS.

In [ ]:
city_areas = cities.area / 1000000 # m2 to km2
city_areas.head()

This indicates that the area of the first city in our table takes up 8.45 square kms. Notice that GeoPandas automatically know to use the `geometry`column for the area calculation.

### Length

An equally common question about lines is their *length*. This computation is straightforward in Python, again provided that our data are projected.
Let's first try a simple length calculation:

In [ ]:
streets.length.head(5)

As you can see, all the streets look very short and GeoPandas is complaining. That is because the streets are in a *geographic* CRS, which means that the street lengths are measured in degrees!
To fix this, we can change the geometries to a projected CRS (`to_crs`) and calculate the length at the same time:

In [ ]:
street_length = streets.to_crs(epsg=25830).length
street_length.head()

Since the CRS we use (`EPSG:25830`) is expressed in metres, we can tell the first street segment is about 370m.

### Centroid calculation

Sometimes it is useful to summarize a polygon into a single point and, for that, a good candidate is its centroid (almost like a spatial analogue of the average). The following command will return a `GeoSeries` (a single column with spatial data) with the centroids of a polygon `GeoDataFrame`:

In [ ]:
cents = cities.centroid
cents.head()

Note how `cents` is not an entire table but a single column, or a `GeoSeries` object. This means you can plot it directly, just like a table:

In [ ]:
cents.plot();

### Buffers

Buffers are one of the classical geospatial operations in which an area is drawn around a particular geometry, based on a chosen radius. These are very useful, for instance, in combination with point-in-polygon operations to calculate accessibility, catchment areas, etc.

For this example, we will use the bars table, but will **reproject** it to the same CRS as `cities`, so it is expressed in metres:

In [ ]:
pois_projected = pois.to_crs(cities.crs)

To create a buffer using `GeoPandas`, simply call the `buffer` method, passing in the radius. For example, to draw a 500m. buffer around every bar in Madrid:

In [ ]:
buf = pois_projected.buffer(500)
buf.head()

Notice how the geometries are now polygons!

Plotting it is equally straighforward:

In [ ]:
f, ax = plt.subplots(1)
# Plot buffer
buf.plot(ax=ax, linewidth=0)
# Plot named places on top for reference
pois_projected.plot(ax=ax, markersize=1, color='yellow');

## FURTHER MATERIALS: Coordinate reference system (CRS) in Geopandas

Luckily, defining and changing projections is easy using Geopandas and a library called [pyproj](https://pyproj4.github.io/pyproj/stable/). In this tutorial we will see how to **retrieve the coordinate reference system information** from the data, and how to **re-project** the data into another crs. We will re-project a data file from
WGS84 (lat, lon coordinates) into a Lambert Azimuthal Equal Area projection which is the [recommended projection for Europe](http://mapref.org/LinkedDocuments/MapProjectionsForEurope-EUR-20120.pdf) by European Commission.

For this tutorial we will be using a Geopackage called `Europe_borders.gpkg` representing the country borders in Europe.
When reading the data into `GeoDataFrame` with Geopandas crs information is automatically stored into the `.crs` attribute of the GeoDataFrame.

In Shapefiles, information about the coordinate reference system is stored in the `.prj` -file. If this file is missing, you might be in trouble!

Let's start by reading the data from the `Europe_borders.gpkg` file and checking the `crs`: 

In [ ]:
# Read the file
fp = "files/europe_borders.gpkg"
data = gpd.read_file(fp)

In [ ]:
# Check the coordinate reference system
data.crs

What we see here is in fact a **CRS object** from the **pyproj** module. 

The EPSG code of our geodataframe is`4326`, which refers to the WGS84 coordinate system.

You can find a lot of information and lists of available coordinate reference systems from:

  - [www.spatialreference.org](http://spatialreference.org/)
  - [www.proj4.org](https://proj4.org/operations/projections/)
  - [www.mapref.org](http://mapref.org/CollectionofCRSinEurope.html)

Let's continue by checking the values in our `geometry`-column to verify that the CRS of our GeoDataFrame seems correct. In the case of EPSG:4326, coordinates should be between -180 and 180° longitude and -90 and 90° latitude.

In [ ]:
data['geometry'].head()

As we can see, the coordinate values of the Polygons indeed look like latitude and longitude values, so everything seems to be in order.

WGS84 projection is not really a good one for representing European borders on a map (areas get distorted), so let's convert those geometries into Lambert Azimuthal Equal Area (LAEA) projection ([EPSG: 3035](http://spatialreference.org/ref/epsg/etrs89-etrs-laea/)) which is the recommended projection by European Comission.

Changing the projection is simple [to do](https://geopandas.org/en/stable/docs/user_guide/projections.html#re-projecting) in GeoPandas with the `.to_crs()`-function, which is a built-in function of the GeoDataFrame. The function has two alternative parameters 1) `crs` and 2) `epgs` that can be used to make the coordinate transformation and re-project the data into the CRS that you want to use.

Let's re-project our data into `EPSG:3035` using the `epsg`-parameter:

In [ ]:
# Let's make a backup copy of our data
data_wgs84 = data.copy()

# Reproject the data
data = data.to_crs(epsg=3035)

# Check the new geometry values
data['geometry'].head()

In [ ]:
data.crs

And here we go, the coordinate values in the geometries have changed! Now we have successfully changed the projection of our layer into a new one, i.e. to `ETRS-LAEA` projection. 

To really understand what is going on, it is good to explore our data visually. Let's compare the datasets by making
maps out of them.


In [ ]:
# Make subplots that are next to each other
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(12, 12))

# Plot the data in WGS84 CRS
data_wgs84.plot(ax=ax1, facecolor='gray')

# Add title
ax1.set_title("WGS84")

# Plot the one with ETRS-LAEA projection
data.plot(ax=ax2, facecolor='blue')

#ax1.set_axis_off()

# Add title
ax2.set_title("ETRS Lambert Azimuthal Equal Area projection")

#ax2.set_axis_off()

# Set aspect ratio as 1
ax1.set_aspect(aspect=1)
ax2.set_aspect(aspect=1)

# Remove empty white space around the plot
plt.tight_layout()

Indeed, the maps look quite different, and the re-projected one looks much better in Europe as the areas especially in the north are more realistic and not so stretched as in WGS84.

Finally, let's save our projected layer so that we can use it later. Note, even if the CRS information is stored in the file, it can be a good idea also to include CRS info in the filename:

In [ ]:
# Save to disk
data.to_file("files/europe_borders_epsg3035.gpkg")

## FURTHER MATERIALS: Global map projections

Finally, let's play around with global map projections. The files folder contains a layer `ne_110m_admin_0_countries.shp` that represents the country borders of the world. The data was downloaded from https://www.naturalearthdata.com/. 

In [ ]:
# Read in data
fp = "files/ne_10m_admin_0_countries/ne_10m_admin_0_countries.shp" 

admin = gpd.read_file(fp)

# Check input crs
admin.crs

In [ ]:
# Set fig size
plt.rcParams['figure.figsize'] = [12, 6]

#Plot in original crs
admin.plot()
plt.title("WGS84");


In [ ]:
# Re-project and plot
admin.to_crs("ESRI:54012").plot()

plt.title("Eckert IV");